In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up paths (same as original code)
BASE_PATH = '/workspace/COMP-3608---PROJECT'
DATA_PATH = os.path.join(BASE_PATH, 'data')
MODELS_PATH = os.path.join(BASE_PATH, 'models')
RESULTS_PATH = os.path.join(BASE_PATH, 'results')

In [ ]:
def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_PATH, f"{title.replace(' ', '_')}.png"))
    plt.show()


In [ ]:
def plot_roc_curve(fpr, tpr, roc_auc, title):
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(RESULTS_PATH, f"{title.replace(' ', '_')}.png"))
    plt.show()

In [ ]:
def print_metrics(y_true, y_pred, model_name):
    """Print and return metrics for a model's predictions"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm
    }

In [ ]:
def ensemble_voting_classifier(model_dict, X_test, y_test, dataset_name, voting='hard'):

    print(f"\n{'-'*50}")
    print(f"Creating Ensemble for {dataset_name} dataset")
    
    model_predictions = {}
    model_probabilities = {}
    
    # Get predictions from each model
    for name, model in model_dict.items():
        model_predictions[name] = model.predict(X_test)
        model_probabilities[name] = model.predict_proba(X_test)[:, 1]
    
    if voting == 'hard':
        # Majority voting (hard voting)
        ensemble_pred = np.zeros(len(y_test))
        for name in model_predictions:
            ensemble_pred += model_predictions[name]
        
        # If more than half of the models predicted 1, set to 1
        ensemble_pred = (ensemble_pred >= len(model_dict)/2).astype(int)
    
    elif voting == 'soft':
        # Averaging probabilities (soft voting)
        ensemble_prob = np.zeros(len(y_test))
        for name in model_probabilities:
            ensemble_prob += model_probabilities[name]
        
        ensemble_prob /= len(model_probabilities)  # average probabilities
        ensemble_pred = (ensemble_prob >= 0.5).astype(int)
    
    # Calculate ensemble metrics
    metrics = print_metrics(y_test, ensemble_pred, f"Ensemble ({voting} voting)")
    
    # Plot confusion matrix
    plot_confusion_matrix(
        metrics['confusion_matrix'], 
        ['Negative', 'Positive'], 
        f'Confusion Matrix - Ensemble {voting.capitalize()} Voting ({dataset_name})'
    )
    
    # Calculate and plot ROC curve (for soft voting)
    if voting == 'soft':
        fpr, tpr, _ = roc_curve(y_test, ensemble_prob)
        roc_auc = auc(fpr, tpr)
        plot_roc_curve(
            fpr, tpr, roc_auc, 
            f'ROC Curve - Ensemble {voting.capitalize()} Voting ({dataset_name})'
        )
    
    return {
        'metrics': metrics,
        'predictions': ensemble_pred,
        'probabilities': ensemble_prob if voting == 'soft' else None
    }

In [ ]:
def compare_with_base_models(ensemble_results, base_model_results, dataset_name):
    """Compare ensemble model with base models"""
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    
    # Extract metrics from base models
    base_metrics = {
        model_name: {metric: base_model_results[model_name][metric] for metric in metrics}
        for model_name in base_model_results
    }
    
    # Add ensemble metrics
    all_metrics = {
        **base_metrics,
        'Ensemble (hard)': {metric: ensemble_results['hard']['metrics'][metric] for metric in metrics},
        'Ensemble (soft)': {metric: ensemble_results['soft']['metrics'][metric] for metric in metrics}
    }
    
    # Create comparison dataframe
    comparison_data = []
    for model_name, model_metrics in all_metrics.items():
        for metric_name, metric_value in model_metrics.items():
            comparison_data.append({
                'Model': model_name,
                'Metric': metric_name,
                'Value': metric_value
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Plot comparison
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i+1)
        metric_data = comparison_df[comparison_df['Metric'] == metric]
        
        sns.barplot(x='Model', y='Value', data=metric_data)
        plt.title(f'{metric.capitalize()} Comparison for {dataset_name}')
        plt.ylim(0, 1)
        plt.xticks(rotation=45)
        plt.tight_layout()
    
    plt.suptitle(f'Model Performance Comparison for {dataset_name}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(os.path.join(RESULTS_PATH, f"{dataset_name}_model_comparison.png"))
    plt.show()
    
    return comparison_df
